In [1]:
# per pathology 
# mIoU ~ AUC, # instances # area

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
import json
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pycocotools import mask
import statsmodels.api as sm
import statsmodels.formula.api as smf
# from eval_helper import iou_seg
import seaborn as sns
from eval_constants import *
from scipy import stats
from eval_constants import *
np.set_printoptions(precision=3)

In [3]:
group_dir = '/deep/group/aihc-bootcamp-spring2020/localize'
output_path = f'{group_dir}/eval_results/regression'

In [7]:
# read vietnam iou
vietnam_dir = f'/deep/group/aihc-bootcamp-spring2020/localize/eval_results/human'
vietnam_iou = pd.read_csv((f'{vietnam_dir}/test_human_ioua_all.csv'))

# read gradcam iou
phase = 'test'
model = 'ensemble'
method = 'gradcam'
iou = pd.read_csv(f'{group_dir}/eval_results/{method}/{phase}_{method}_{model}_merged_iou_prob_threshold.csv')

# pt game vietnam and gradcam
pt_vietnam_df = pd.read_csv((f'{group_dir}/eval_results/human/pt_vietnam.csv'),index_col = False)
pt_gradcam = pd.read_csv(f'{group_dir}/eval_results/gradcam/Pointing_game_gradcam_new.csv')

# read geometric features
group_dir = '/deep/group/aihc-bootcamp-spring2020/localize'
regression_path = f'{group_dir}/eval_results/regression'

instance_df = pd.read_csv(f'{regression_path}/num_instances_test.csv').drop(['img_id'],axis=1)
areas_df = pd.read_csv(f'{regression_path}/area_ratio_test.csv').drop(['img_id'],axis=1)
elongation_df = pd.read_csv('elogation.csv').drop(['img_id','Unnamed: 0'],axis=1)
rec_area_ratio_df = pd.read_csv('rec_area_ratio.csv').drop(['img_id','Unnamed: 0'],axis=1)
rec_area_ratio_df = 1-rec_area_ratio_df
confidence_df = pd.read_csv(f"{group_dir}/annotations/difficulty.csv")
# read probabitlies 
probs_df = pd.read_csv(f'{group_dir}/eval_results/prob.csv')

In [5]:
# create regression dataframe 

for task in sorted(LOCALIZATION_TASKS):

    df = pd.DataFrame()
    
    data = {'iou':iou[task].values,
            'human_iou':vietnam_iou[task].values,
            'iou_diff': vietnam_iou[task].values - iou[task].values,
            'gradcam_pt': pt_gradcam[task].values,
            'human_pt': pt_vietnam_df[task].values,
            'pt_diff':  pt_vietnam_df[task].values - pt_gradcam[task].values ,
            'n_instance': instance_df[task].tolist(),
            'area_ratio':areas_df[task].tolist(),
            'elongation': elongation_df[task].tolist(),
            'rec_area_ratio': rec_area_ratio_df[task].tolist(),
            'difficulty': confidence_df[task].tolist(),
            'prob': probs_df[task].tolist()
           }
    df = pd.DataFrame(data) 
    df = df[df.n_instance>0]
        
    df.to_csv(f'{regression_path}/regression_{task}.csv', index = False)

In [4]:
def normalize(column):
    if column.min() == column.max():
        return column
    
    return (column-column.min())/(column.max()-column.min())

In [5]:
def standardize(column):
    if column.min() == column.max():
        return column
    
    return (column-column.mean())/(column.std())

In [6]:
coef_summary = pd.DataFrame(columns = ["lower","upper","mean","coef_pval","corr","corr_pval","feature","task"])
y = 'gradcam_pt'

features = ['n_instance','area_ratio','elongation','rec_area_ratio']

for task in LOCALIZATION_TASKS:
    print(task)
    regression_df = pd.read_csv(f'{regression_path}/regression_{task}.csv')
    for feature in features:
        # normalize feature
        regression_df[feature] = standardize(regression_df[feature])

        # run regression
        est = smf.ols(f"{y} ~ {feature}", data = regression_df)
        est2 = est.fit()
        ci = est2.conf_int(alpha=0.05, cols=None)  # get ci 
        lower,upper = ci.loc[feature]
        mean = est2.params.loc[feature]
        pval = est2.pvalues.loc[feature]
        corr, corr_pval = stats.spearmanr(regression_df[y].values,regression_df[feature].values,nan_policy = 'omit')
#         corr = regression_df[y].corr(regression_df[feature], method='spearman')

        # append to dataframe
        coef_summary = coef_summary.append({'lower': lower,
                                            'upper': upper,
                                            'mean': mean,
                                            'coef_pval': pval,
                                            'corr_pval': corr_pval,
                                            'corr': corr,
                                            'n': len(regression_df),
                                            'feature': feature,
                                            'task': task},ignore_index=True)
        

# coef_summary.to_csv(f'tables/regression_{y}.csv')

Enlarged Cardiomediastinum
Cardiomegaly


/sailhome/xgui/deep/u/xgui/envs/chexpert-baseline/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/sailhome/xgui/deep/u/xgui/envs/chexpert-baseline/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/sailhome/xgui/deep/u/xgui/envs/chexpert-baseline/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/sailhome/xgui/deep/u/xgui/envs/chexpert-baseline/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/sailhome/xgui/deep/u/xgui/envs/chexpert-baseline/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


Lung Lesion
Airspace Opacity
Edema
Consolidation
Atelectasis
Pneumothorax
Pleural Effusion
Support Devices


In [12]:
coef_summary.to_csv('reg_coef_by_task.csv')

In [8]:
import math
overall_regression = pd.DataFrame()
for task in sorted(LOCALIZATION_TASKS):
    df_method = pd.read_csv(f'{regression_path}/regression_{task}.csv')
    df_method['task'] = task
    overall_regression = overall_regression.append(df_method)
# overall_regression.to_csv(f'{regression_path}/overall_regression.csv', index = False)

In [10]:
overall_coef = pd.DataFrame(columns = ["lower","upper","mean","coef_pval","corr","corr_pval","corr_lower","corr_upper","feature"])
y = 'human_pt'
features = ['n_instance','area_ratio','elongation','rec_area_ratio']

for feature in features:
    # normalize feature
    overall_regression[feature] = normalize(overall_regression[feature])

    # run regression
    est = smf.ols(f"{y} ~ {feature}", data = overall_regression)
    est2 = est.fit()
    ci = est2.conf_int(alpha=0.05, cols=None)  # get ci 
    lower,upper = ci.loc[feature]
    mean = est2.params.loc[feature]
#     corr = overall_regression[y].corr(overall_regression[feature], method='spearman')
    pval = est2.pvalues.loc[feature]
    corr, corr_pval = stats.spearmanr(overall_regression[feature].values,overall_regression[y].values)
    
    n = len(overall_regression)
    stderr = 1.0 / math.sqrt(n - 3)
    delta = 1.96 * stderr
    lower_r = math.tanh(math.atanh(corr) - delta)
    upper_r = math.tanh(math.atanh(corr) + delta)
    # append to dataframe
    overall_coef = overall_coef.append({'lower': lower,
                                        'upper': upper,
                                        'mean': mean,
                                        'coef_pval': pval,
                                        'corr': corr,
                                        'corr_pval': corr_pval,
                                        'corr_lower': lower_r,
                                        'corr_upper': upper_r,
                                        'feature': feature},ignore_index=True)
print(overall_coef)
# overall_coef.to_csv(f'tables_final/{y}_coef_overall.csv')

      lower     upper      mean     coef_pval      corr     corr_pval  \
0  0.111852  0.694621  0.403236  5.538308e-04  0.088420  5.262092e-04   
1  0.177545  0.448210  0.312877  8.966260e-09  0.148258  5.403333e-09   
2  0.061208  0.471982  0.266595  1.196140e-03  0.105872  3.254052e-05   
3  0.062351  0.274990  0.168670  7.608766e-05  0.088433  5.252058e-04   

   corr_lower  corr_upper         feature  
0    0.038540    0.137860      n_instance  
1    0.098942    0.196847      area_ratio  
2    0.056119    0.155100      elongation  
3    0.038553    0.137873  rec_area_ratio  


In [21]:
overall_coef.round(3).to_csv("tables/human_pt_coef_overall.csv")

### Regression on Probability

In [29]:
overall_coef

,lower,upper,mean,coef_pval,corr,corr_pval,feature
0,0.030595,0.219479,0.125037,9.495926e-03,0.063851,0.911024,difficulty
1,-1.307028,-0.714032,-1.010530,3.219130e-11,-0.165435,0.276871,n_instance
2,0.981659,1.240028,1.110844,1.111863e-58,0.408794,0.361419,area_ratio
3,-1.049714,-0.634363,-0.842038,3.503033e-15,-0.163746,0.099856,elongation
4,-0.642267,-0.429157,-0.535712,2.768960e-22,-0.192051,0.540695,rec_area_ratio


In [108]:
for task in sorted(LOCALIZATION_TASKS):
    df = pd.DataFrame()
    
    data = {'iou':iou[task].values,
            'pt': pt_gradcam[task].values,
            'prob': probs_df[task].tolist()
           }
    df = pd.DataFrame(data) 
    df = df[df.iou>-1]
        
    df.to_csv(f'{regression_path}/regression_prob_{task}.csv', index = False)

In [73]:
iou.describe()

,Airspace Opacity,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Lung Lesion,Pleural Effusion,Pneumothorax,Support Devices
count,381.000000,296.000000,229.000000,120.000000,124.000000,668.000000,50.000000,159.000000,11.000000,327.000000
mean,0.192792,0.144632,0.276150,0.087344,0.174317,0.168751,0.027214,0.157548,0.125245,0.142708
std,0.182429,0.166890,0.260783,0.185545,0.206641,0.218174,0.068903,0.180373,0.136617,0.092345
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.094873
50%,0.177769,0.060922,0.343475,0.000000,0.000000,0.000000,0.000000,0.091621,0.080243,0.137904
75%,0.339432,0.270177,0.498283,0.000000,0.364922,0.364307,0.000000,0.278672,0.228878,0.194682
max,0.676121,0.601912,0.774007,0.641519,0.632902,0.792309,0.280265,0.713100,0.350111,0.466935


In [84]:
overall_prob_regression = pd.DataFrame()
for task in sorted(LOCALIZATION_TASKS):
    df_method = pd.read_csv(f'{regression_path}/regression_prob_{task}.csv')
    df_method['task'] = task
    overall_prob_regression = overall_prob_regression.append(df_method)
overall_prob_regression.to_csv(f'{regression_path}/overall_prob_regression.csv', index = False)

In [131]:
len(overall_prob_regression)

2365

In [93]:
overall_coef = pd.DataFrame(columns = ["lower","upper","mean","coef_pval","corr","corr_pval","corr_lower","corr_upper","task",'n'])
y = 'pt'
feature = 'prob'

# overall_prob_regression[feature] = normalize(overall_prob_regression[feature])

# run regression
est = smf.ols(f"{y} ~ {feature}", data = overall_prob_regression)
est2 = est.fit()
ci = est2.conf_int(alpha=0.05, cols=None)  # get ci 
lower,upper = ci.loc[feature]
mean = est2.params.loc[feature]
pval = est2.pvalues.loc[feature]
corr, corr_pval = stats.spearmanr(overall_prob_regression[feature].values,overall_prob_regression[y].values,nan_policy = 'omit')
n = len(overall_regression)
stderr = 1.0 / math.sqrt(n - 3)
delta = 1.96 * stderr
lower_r = math.tanh(math.atanh(corr) - delta)
upper_r = math.tanh(math.atanh(corr) + delta)
# append to dataframe
overall_coef = overall_coef.append({'lower': lower,
                                    'upper': upper,
                                    'mean': mean,
                                    'coef_pval': pval,
                                    'corr': corr,
                                    'corr_pval': corr_pval,
                                    'corr_lower': lower_r,
                                    'corr_upper':upper_r,
                                    'task': "Overall",
                                   'n': len(overall_prob_regression)},ignore_index=True)
overall_coef.to_csv(f'tables/overall_regression_prob_{y}.csv')

In [133]:
overall_coef.round(3)

,lower,upper,mean,coef_pval,corr,corr_pval,task,n
0,0.083,0.135,0.109,0.0,0.285,0.0,Overall,2365


In [86]:
overall_coef.round(3)

,lower,upper,mean,coef_pval,corr,corr_pval,corr_lower,corr_upper,task,n
0,0.083,0.135,0.109,0.0,0.285,0.0,0.239,0.331,Overall,2365


In [92]:
coef_summary = pd.DataFrame(columns = ["lower","upper","mean","coef_pval","corr","corr_pval","corr_lower", "corr_upper", "feature","task"])
y = 'pt'

feature = 'prob'

for task in LOCALIZATION_TASKS:
    print(task)
    regression_df = pd.read_csv(f'{regression_path}/regression_prob_{task}.csv')
   
    # normalize feature
#     regression_df[feature] = standardize(regression_df[feature])

    # run regression
    est = smf.ols(f"{y} ~ {feature}", data = regression_df)
    est2 = est.fit()
    ci = est2.conf_int(alpha=0.05, cols=None)  # get ci 
    lower,upper = ci.loc[feature]
    mean = est2.params.loc[feature]
    pval = est2.pvalues.loc[feature]
    corr, corr_pval = stats.spearmanr(regression_df[y].values,regression_df[feature].values,nan_policy = 'omit')
    n = len(overall_regression)
    stderr = 1.0 / math.sqrt(n - 3)
    delta = 1.96 * stderr
    lower_r = math.tanh(math.atanh(corr) - delta)
    upper_r = math.tanh(math.atanh(corr) + delta)

    # append to dataframe
    coef_summary = coef_summary.append({'lower': lower,
                                        'upper': upper,
                                        'mean': mean,
                                        'coef_pval': pval,
                                        'corr_pval': corr_pval,
                                        'corr': corr,
                                        'corr_lower': lower_r,
                                        'corr_upper':upper_r,
                                        'n': len(regression_df),
                                        'feature': feature,
                                        'task': task},ignore_index=True)

# coef_summary.to_csv(f'tables/regression_prob_{y}.csv')

Enlarged Cardiomediastinum
Cardiomegaly
Lung Lesion
Airspace Opacity
Edema
Consolidation
Atelectasis
Pneumothorax
Pleural Effusion
Support Devices


In [136]:
%precision 3
coef_summary.round(3)[['mean','coef_pval','corr','corr_pval','task','n']]#.to_csv('prob_experiments_results.csv')

,mean,coef_pval,corr,corr_pval,task,n
0,1.974,0.000,0.428,0.00,Enlarged Cardiomediastinum,668.0
1,0.679,0.000,0.592,0.00,Cardiomegaly,229.0
2,0.218,0.002,0.509,0.00,Lung Lesion,50.0
3,0.714,0.000,0.577,0.00,Airspace Opacity,381.0
4,0.642,0.000,0.548,0.00,Edema,124.0
5,1.155,0.000,0.384,0.00,Consolidation,120.0
6,0.489,0.000,0.348,0.00,Atelectasis,296.0
7,0.446,0.015,0.734,0.01,Pneumothorax,11.0
8,0.632,0.000,0.690,0.00,Pleural Effusion,159.0
9,0.211,0.000,0.468,0.00,Support Devices,327.0
